In [1]:
import pyspark.sql.functions as f

In [2]:
spark

In [3]:
df = spark.read.json("/home/ubuntu/temp-data")
df.show()

+--------------+----+-------------------+----+--------------------+---------------+
|extractionTime| geo|                 id|lang|                text|           user|
+--------------+----+-------------------+----+--------------------+---------------+
|    1639929240|null|1472596084853600263|  hi|@JoeBiden उत्तर प...|   mayankshukl1|
|    1639929328|null|1472596451905531905| und|@Sen_JoeManchin ?...|    snowstars22|
|    1639929571|null|1472597472530681860|  en|@Sen_JoeManchin ?...| mindfulliving1|
|    1639929243|null|1472596097419730946|  en|@Sen_JoeManchin  ...|    FacedSchiff|
|    1639929309|null|1472596372654276610|  en|@Sen_JoeManchin “...|       EricAEPA|
|    1639929253|null|1472596139442688011|  en|@Sen_JoeManchin “...|       EricAEPA|
|    1639929571|null|1472597472597979137|  en|@Sen_JoeManchin P...|       MWoody29|
|    1639929284|null|1472596268430118916|  en|RT @TheDemocrats:...|   JKim30498725|
|    1639929254|null|1472596143897034752|  en|RT @TheDemocrats:...|    lilma

In [8]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [12]:
df.select(f.explode(f.split("text", " ")).alias("word"))\
.filter(~f.lower("word").isin(stopwords))\
.groupby("word").count().orderBy(f.col("count").desc()).show()

+----------------+-----+
|            word|count|
+----------------+-----+
| @Sen_JoeManchin|  149|
|              RT|   79|
|                |   33|
|          @POTUS|   29|
|             one|   25|
|     @SenSchumer|   21|
|@LindseyGrahamSC|   19|
|       @JoeBiden|   17|
|               .|   15|
|          people|   14|
|           going|   13|
|          Senate|   12|
|  @amyklobuchar:|   11|
|            must|   11|
|            Cruz|   11|
|         federal|   11|
|          Thanks|   11|
|          agreed|   10|
|           thro…|   10|
|         session|   10|
+----------------+-----+
only showing top 20 rows



In [7]:
df.count()

336